In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.base import clone
import lightgbm as lgb
from tqdm import tqdm

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import geohash2 as geohash


train_path = "C:\\Users\\USER\\Desktop\\House price\\dataset.csv"
test_path = "C:\\Users\\USER\\Desktop\\House price\\test.csv"
sam_path = "C:\\Users\\USER\\Desktop\\House price\\sample_submission.csv"

print("Libraries installed successfully!")

Libraries installed successfully!


import data

In [2]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_df = pd.read_csv(sam_path)

print(train_df.shape)
print(train_df.columns)
train_df.head()

(200000, 47)
Index(['id', 'sale_date', 'sale_price', 'sale_nbr', 'sale_warning',
       'join_status', 'join_year', 'latitude', 'longitude', 'area', 'city',
       'zoning', 'subdivision', 'present_use', 'land_val', 'imp_val',
       'year_built', 'year_reno', 'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt',
       'grade', 'fbsmt_grade', 'condition', 'stories', 'beds', 'bath_full',
       'bath_3qtr', 'bath_half', 'garb_sqft', 'gara_sqft', 'wfnt', 'golf',
       'greenbelt', 'noise_traffic', 'view_rainier', 'view_olympics',
       'view_cascades', 'view_territorial', 'view_skyline', 'view_sound',
       'view_lakewash', 'view_lakesamm', 'view_otherwater', 'view_other',
       'submarket'],
      dtype='object')


,id,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,...,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
0,0,2014-11-15,236000,2.0,,nochg,2025,47.2917,-122.3658,53,...,0,0,0,0,0,0,0,0,0,I
1,1,1999-01-15,313300,NaN,26,nochg,2025,47.6531,-122.1996,74,...,0,0,0,0,0,1,0,0,0,Q
2,2,2006-08-15,341000,1.0,,nochg,2025,47.4733,-122.1901,30,...,0,0,0,0,0,0,0,0,0,K
3,3,1999-12-15,267000,1.0,,nochg,2025,47.4739,-122.3295,96,...,0,0,0,0,0,0,0,0,0,G
4,4,2018-07-15,1650000,2.0,,miss99,2025,47.7516,-122.1222,36,...,0,0,0,0,0,0,0,0,0,P


In [3]:
#計算缺失值
missing_values = train_df.isnull().sum()
missing_values[missing_values > 0].sort_values(ascending=False)

sale_nbr       42182
subdivision    17550
submarket       1717
dtype: int64

feature engineering

In [ ]:
def zoning_group_classify(z):
    if pd.isna(z): return 'other'
    z = z.upper()
    if 'SF' in z: return 'SF'
    elif 'MR' in z: return 'MR'
    elif 'NC' in z: return 'NC'
    elif 'P' in z: return 'P'
    elif 'HR' in z or 'IG' in z: return 'other'
    return 'other'

def encode_dataset(df, is_train=True, top_cities=None, top_supermarket=None, top_sale_warning=None):
    
    encoded = pd.DataFrame()
    confirmed_one_hot = ['join_status', 'condition', 'stories', 'grade', 'fbsmt_grade', 'present_use']

    direct_add_cols = [
        'id', 'sale_price', 'join_year', 'latitude', 'longitude',
        'area', 'land_val', 'imp_val', 'year_built', 'year_reno',
        'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt',
        'beds', 'garb_sqft', 'gara_sqft', 'golf', 'greenbelt',
        'bath_full', 'bath_3qtr', 'bath_half', 'wfnt', 'noise_traffic',
        'view_rainier', 'view_olympics', 'view_cascades', 'view_territorial',
        'view_skyline', 'view_sound', 'view_lakewash', 'view_lakesamm',
        'view_otherwater', 'view_other'
        #'subdivision','sale_nbr'沒有做這個 用意不大
    ]

    onehot_df = pd.get_dummies(df[confirmed_one_hot], drop_first=False)
    encoded = pd.concat([encoded, onehot_df], axis=1)

    df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
    encoded['sale_year'] = df['sale_date'].dt.year
    encoded['sale_month'] = df['sale_date'].dt.month
    encoded['sale_season'] = ((encoded['sale_month'] - 1) // 3 + 1)

    for col in direct_add_cols:
        if col in df.columns:
            encoded[col] = df[col]

    if is_train:
        top_cities = df['city'].value_counts().nlargest(10).index.tolist()
        top_supermarket = df['submarket'].value_counts().nlargest(10).index.tolist()
        top_sale_warning = df['sale_warning'].value_counts().nlargest(15).index.tolist()

    encoded['city_simplified'] = df['city'].apply(lambda x: x if x in top_cities else 'other')
    encoded['submarket_simplified'] = df['submarket'].apply(lambda x: x if x in top_supermarket else 'other')
    encoded['sale_warning_simplified'] = df['sale_warning'].apply(lambda x: x if x in top_sale_warning else 'other')

    city_dummy = pd.get_dummies(encoded['city_simplified'], prefix='city', drop_first=False)
    submarket_dummy = pd.get_dummies(encoded['submarket_simplified'], prefix='submarket', drop_first=False)
    sale_warning_dummy = pd.get_dummies(encoded['sale_warning_simplified'], prefix='sale_warning', drop_first=False)
    encoded = pd.concat([encoded, city_dummy, submarket_dummy, sale_warning_dummy], axis=1)

    encoded['zoning_group'] = df['zoning'].apply(zoning_group_classify)
    zoning_dummy = pd.get_dummies(encoded['zoning_group'], prefix='zoning_group', drop_first=False)
    encoded = pd.concat([encoded, zoning_dummy], axis=1)
    encoded.drop(columns=['zoning_group', 'city_simplified', 'submarket_simplified', 'sale_warning_simplified'], inplace=True)

    encoded['age'] = encoded['sale_year'] - encoded['year_built']
    encoded['renovated'] = np.where(encoded['year_reno'] > 0, 1, 0)
    encoded['years_since_reno'] = np.where(encoded['renovated'], encoded['sale_year'] - encoded['year_reno'], 0)
    encoded['total_baths'] = encoded['bath_full'] + 0.75 * encoded['bath_3qtr'] + 0.5 * encoded['bath_half']
    encoded['total_value'] = encoded['land_val'] + encoded['imp_val']
    encoded['living_area'] = encoded['sqft'] + encoded['sqft_fbsmt']


    encoded["floor_ratio"] = np.where(
    encoded["sqft_lot"] == 0,
    0,
    encoded["sqft"] / encoded["sqft_lot"]
    )

    encoded["is_large_house"] = (encoded["sqft"] > 3000).astype(int)
    encoded["is_recent_reno"] = (encoded["years_since_reno"] <= 5).astype(int)
    encoded["bath_per_bed"] = encoded["total_baths"] / encoded["beds"]
    encoded["bath_per_bed"] = encoded["bath_per_bed"].replace([np.inf, -np.inf], 0).fillna(0)
    
    return encoded, top_cities, top_supermarket, top_sale_warning

'''def pca_train_test(encoded, feature, scaler=None, pca=None, kmeans=None):
    """
    如果傳入 scaler/pca/kmeans == None → 在 df 上 fit
    否則只做 transform / predict
    回傳 (encoded, scaler, pca, kmeans)
    """
    # 1. 標準化
    if scaler is None:
        scaler = StandardScaler().fit(encoded[feature])
    X_scaled = scaler.transform(encoded[feature])

    # 2. PCA
    if pca is None:
        pca = PCA(n_components=3, random_state=42).fit(X_scaled)
    X_pca = pca.transform(X_scaled)

    # 3. KMeans
    if kmeans is None:
        kmeans = KMeans(n_clusters=10, random_state=42).fit(X_pca)
    encoded["pca_region_cluster"] = kmeans.predict(X_pca)

    # 4. One‑hot
    dummies = pd.get_dummies(encoded["pca_region_cluster"], prefix="pca_region")
    encoded = pd.concat([encoded, dummies], axis=1)
    encoded.drop(columns=["pca_region_cluster"], inplace=True)

    return encoded, scaler, pca, kmeans'''



def clean_features(df, log_cols=None, clip_cols=None, add_log_target=False, log_target_col="sale_price_log"):
    """
    對 df 做對數化 / clip。
    Parameters
    ----------
    df : pd.DataFrame
    log_cols : list[str]  要 log1p 的欄位。預設常見數值長尾欄。
    clip_cols : list[str] 要 clip 的欄位（可 None）。
    add_log_target : bool  是否另外產生 log 版目標欄（不覆寫原欄位）。
    log_target_col : str   新增目標欄名稱。
    Returns
    -------
    cleaned_df : pd.DataFrame
    """
    df = df.copy()  # 不汙染呼叫端

    # 1️⃣ 預設欄位表
    if log_cols is None:
        log_cols = ['land_val', 'imp_val', 'sqft_lot',
                    'garb_sqft', 'floor_ratio', 'total_value']
    if clip_cols is None:
        clip_cols = ['land_val', 'imp_val', 'sqft_lot']

    # 2️⃣ log1p
    for col in log_cols:
        if col in df.columns:
            df[col] = np.log1p(df[col])

    # 3️⃣ clip（可選）
    for col in clip_cols:
        if col in df.columns:
            df[col] = df[col].clip(upper=1_000_000)

    # 4️⃣ 目標值也要 log（加新欄，不覆蓋）
    if add_log_target and 'sale_price' in df.columns:
        df[log_target_col] = np.log1p(df['sale_price'])

    return df

def add_geohash(df, prec=6):
    df = df.copy()
    df['gh6'] = df.apply(
        lambda r: geohash.encode(r.latitude, r.longitude, precision=prec),
        axis=1
    )
    return df


In [19]:
# 1️⃣ 產生 base features
train_encoded, top_cities, top_supermarket, top_sale_warning = encode_dataset(train_df, is_train=True)
test_encoded , _ , _ , _  = encode_dataset(test_df , is_train=False,
                                           top_cities=top_cities,
                                           top_supermarket=top_supermarket,
                                           top_sale_warning=top_sale_warning)

'''# 2️⃣ PCA + KMeans：train 先 fit，test 只 transform
pca_features = ['latitude', 'longitude', 'sqft', 'area', 'total_value', 'imp_val']
train_encoded, scaler, pca, kmeans = pca_train_test(train_encoded, pca_features)
test_encoded , _,     _,   _       = pca_train_test(test_encoded , pca_features,
                                                    scaler=scaler, pca=pca, kmeans=kmeans)'''

# geohash 6 
train_encoded = add_geohash(train_encoded)
test_encoded  = add_geohash(test_encoded)

# ---------- TRAIN ----------
gh_train_dum = pd.get_dummies(train_encoded['gh6'], prefix='gh6', drop_first=False)
train_encoded = pd.concat([train_encoded, gh_train_dum], axis=1)
train_encoded.drop(columns=['gh6'], inplace=True)    # 原 gh6 類別欄可丟

# ---------- TEST ----------
gh_test_dum = pd.get_dummies(test_encoded['gh6'], prefix='gh6', drop_first=False)
test_encoded = pd.concat([test_encoded, gh_test_dum], axis=1)
test_encoded.drop(columns=['gh6'], inplace=True)

# 3️⃣ 其餘特徵工程
train_encoded = clean_features(train_encoded, add_log_target=True)
test_encoded  = clean_features(test_encoded , add_log_target=False)

train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)

In [20]:
#確認資料類型
train_encoded.dtypes.value_counts()

bool       3891
int64        35
float64      12
int32         6
Name: count, dtype: int64

LGBM Quantile

In [21]:
'''# 分割訓練特徵與目標
X = train_encoded.drop(columns=['sale_price', 'id'])  # id 可留給最後輸出
y = train_encoded['sale_price']'''

# 👉 用 log(price) 當 y
X = train_encoded.drop(columns=['sale_price', 'sale_price_log', 'id'])
y = train_encoded['sale_price_log']
y_raw = np.expm1(y)

In [22]:
def winkler_score(y_true, lower, upper, alpha=0.1):
    width = upper - lower
    below = np.maximum(lower - y_true, 0)
    above = np.maximum(y_true - upper, 0)
    return width + (2 / alpha) * (below + above)

def oof_and_hill_climb_two_weights(X, y, model_lower, model_upper, alpha=0.1, n_splits=5, seed=42, steps=100):

    y_raw = np.expm1(y)

    oof_lowers = np.zeros(len(X))
    oof_uppers = np.zeros(len(X))

    fold_lower_models = []
    fold_upper_models = []

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        lower_model = clone(model_lower)
        upper_model = clone(model_upper)

        callbacks = [
        lgb.early_stopping(stopping_rounds=200),
        lgb.log_evaluation(period=0)
        ]


        lower_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="quantile",
        callbacks=callbacks
        )

        upper_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="quantile",
        callbacks=callbacks
        )

        #oof_lowers[val_idx] = lower_model.predict(X_val)
        #oof_uppers[val_idx] = upper_model.predict(X_val)

        # － lower
        oof_lowers[val_idx] = np.expm1(   # 👈 還原
        lower_model.predict(X_val, num_iteration=lower_model.best_iteration_)
        )
        # － upper
        oof_uppers[val_idx] = np.expm1(   # 👈 還原
        upper_model.predict(X_val, num_iteration=upper_model.best_iteration_)
        )

        fold_lower_models.append(lower_model)
        fold_upper_models.append(upper_model)


    # 初始化雙權重
    current_w1 = 0.4  # 下限 weight
    current_w2 = 0.6  # 上限 weight

    best_score = np.inf
    best_weights = (current_w1, current_w2)

    '''for step in range(steps):
        # 微調 perturbation，讓 weight 有隨機性（避免卡住）
        perturb1 = np.random.dirichlet([9])[0] - 0.9
        perturb2 = np.random.dirichlet([9])[0] - 0.9

        w1 = np.clip(current_w1 + 0.1 * perturb1, 0, 1)
        w2 = np.clip(current_w2 + 0.1 * perturb2, 0, 1)

        # 雙權重組合
        lower_combined = w1 * oof_lowers + (1 - w1) * oof_uppers
        upper_combined = w2 * oof_uppers + (1 - w2) * oof_lowers

        # 修正：確保上下限方向正確（防止預測範圍錯位）
        lower_combined, upper_combined = np.minimum(lower_combined, upper_combined), np.maximum(lower_combined, upper_combined)

        score = np.mean(winkler_score(y_raw, lower_combined, upper_combined, alpha))

        if score < best_score:
            best_score = score
            best_weights = (w1, w2)
            current_w1, current_w2 = w1, w2
            print(f"[Step {step}] ✅ Improved Score: {best_score:.2f} (w1: {w1:.4f}, w2: {w2:.4f})")'''
    


    grid = np.linspace(0, 1, 401)          # 0.0025 步
    best_score = np.inf
    best_weights = (0, 0)
    best_cov = 0

    y_raw = np.expm1(y)                    # 還原單位一次就好

    for w1 in grid:
        for w2 in grid:
            if w1 > w2:                       # 保持下限≤上限
                continue
            low  = w1 * oof_lowers + (1 - w1) * oof_uppers
            high = w2 * oof_uppers + (1 - w2) * oof_lowers

            score = winkler_score(y_raw, low, high, alpha).mean()
            if score < best_score:
                best_score = score
                best_weights = (w1, w2)
                best_cov = ((y_raw >= low) & (y_raw <= high)).mean()

    print(f"[Grid] best Winkler {best_score:.0f}  "
          f"w1={best_weights[0]:.3f}  w2={best_weights[1]:.3f}  "
          f"cov={best_cov:.3f}")

    return (oof_lowers, oof_uppers, 
            best_weights, best_score , 
            fold_lower_models, fold_upper_models)

In [23]:
models = {
    "lower": lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.05,
        device="cpu",
        n_estimators=8000,
        learning_rate=0.03,
        num_leaves=63,
        subsample=0.8,
        subsample_freq=1,
        random_state=42
    ),
    "upper": lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.95,
        device="cpu",
        n_estimators=8000,
        learning_rate=0.03,
        num_leaves=63,
        subsample=0.8,
        subsample_freq=1,
        random_state=42
    )
}

In [24]:
oof_lowers, oof_uppers, best_weights, best_score , fold_lower_models, fold_upper_models= oof_and_hill_climb_two_weights(
    X, y,
    model_lower=models["lower"],
    model_upper=models["upper"],
    alpha=0.1, 
    n_splits=5,
    steps=100
)

w1_opt, w2_opt = best_weights
print(f"OOF Winkler  = {best_score:.0f}")
print(f"最佳權重      = w1 {w1_opt:.3f} / w2 {w2_opt:.3f}")

'''grid = np.linspace(0, 1, 401)
best = (np.inf, 0, 0, 0)                   # (winkler, w1, w2, coverage)

for w1 in grid:
    for w2 in grid:
        if w1 <= w2:
            lower = w1 * oof_lowers + (1 - w1) * oof_uppers
            upper = w2 * oof_uppers + (1 - w2) * oof_lowers

            cov   = ((y_raw >= lower) & (y_raw <= upper)).mean()
            
            score = winkler_score(y_raw, lower, upper).mean()
            if score < best[0]:
                best = (score, w1, w2, cov)

print(f"best Winkler {best[0]:.0f}  w1={best[1]:.3f}  w2={best[2]:.3f}  cov={best[3]:.3f}")'''

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.295253 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8040
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 2022
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 12.128117
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1714]	valid_0's quantile: 0.0152108
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.284303 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wi

'grid = np.linspace(0, 1, 401)\nbest = (np.inf, 0, 0, 0)                   # (winkler, w1, w2, coverage)\n\nfor w1 in grid:\n    for w2 in grid:\n        if w1 <= w2:\n            lower = w1 * oof_lowers + (1 - w1) * oof_uppers\n            upper = w2 * oof_uppers + (1 - w2) * oof_lowers\n\n            cov   = ((y_raw >= lower) & (y_raw <= upper)).mean()\n            \n            score = winkler_score(y_raw, lower, upper).mean()\n            if score < best[0]:\n                best = (score, w1, w2, cov)\n\nprint(f"best Winkler {best[0]:.0f}  w1={best[1]:.3f}  w2={best[2]:.3f}  cov={best[3]:.3f}")'

before plus floor ratio~bath per bed score = 341552  (original type)
after plus new feature score = 342013
after adjust model = 387816

try to check coverage = 340265.61 (I put new feature and do pca but not doing stacking model) ##model alpha set as 0.05 & 0.95
try to check coverage = 337115.51 (I put new feature and do pca but not doing stacking model) ##model alpha set as 0.05 & 0.95
instead pca&kmean by geohash = 336938

離線測試

In [25]:
# 先把真實價格還原到『元』
y_raw = np.expm1(y)

# 5 個模型各自預測整個 X，再平均
oof_low_mean  = np.zeros(len(X))
oof_high_mean = np.zeros(len(X))

for lo, up in zip(fold_lower_models, fold_upper_models):
    oof_low_mean  += np.expm1(lo.predict(X, num_iteration=lo.best_iteration_))
    oof_high_mean += np.expm1(up.predict(X, num_iteration=up.best_iteration_))
oof_low_mean  /= len(fold_lower_models)
oof_high_mean /= len(fold_upper_models)

# 套最佳權重
low_final  = w1_opt * oof_low_mean  + (1 - w1_opt) * oof_high_mean
high_final = w2_opt * oof_high_mean + (1 - w2_opt) * oof_low_mean
low_final, high_final = np.minimum(low_final, high_final), np.maximum(low_final, high_final)

# 計 coverage 與 Winkler
cov_final = ((y_raw >= low_final) & (y_raw <= high_final)).mean()
wink_final = winkler_score(y_raw, low_final, high_final, alpha=0.1).mean()

print(f"5‑fold 平均 + 權重  ->  Winkler {wink_final:.0f} ,  coverage {cov_final:.3f}")

5‑fold 平均 + 權重  ->  Winkler 265830 ,  coverage 0.882


輸出test LGBM

In [26]:
#填補缺漏欄位（對齊訓練集欄位）
missing_cols = set(X.columns) - set(test_encoded.columns)
for col in missing_cols:
    test_encoded[col] = 0

# 確保欄位順序一致
test_encoded = test_encoded[X.columns]

In [27]:
# 先確保 test_encoded 欄位與 X 完全一致
test_encoded = test_encoded[X.columns]

# ---------- 5‑fold 平均預測 ----------
pred_low_raw  = np.zeros(len(test_encoded))
pred_high_raw = np.zeros(len(test_encoded))

for lo, up in zip(fold_lower_models, fold_upper_models):          # ← 名稱跟 return 一致
    pred_low_raw  += np.expm1(lo.predict(test_encoded,  num_iteration=lo.best_iteration_))
    pred_high_raw += np.expm1(up.predict(test_encoded, num_iteration=up.best_iteration_))

pred_low_raw  /= len(fold_lower_models)
pred_high_raw /= len(fold_upper_models)

# ---------- 套用最佳權重 ----------
test_lower = w1_opt * pred_low_raw  + (1 - w1_opt) * pred_high_raw
test_upper = w2_opt * pred_high_raw + (1 - w2_opt) * pred_low_raw
test_lower, test_upper = np.minimum(test_lower, test_upper), np.maximum(test_lower, test_upper)

In [28]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df.head()
test_encoded['id'] = test_df['id']  # 這行先補上 id

In [29]:
submission_df = pd.DataFrame({
    'id': test_encoded['id'],  # 必須與 sample_submission 對齊
    'pi_lower': test_lower,
    'pi_upper': test_upper
})

# 輸出成 CSV
submission_df.to_csv('geo6_5fold.csv', index=False)
print(submission_df.head())

       id       pi_lower      pi_upper
0  200000  822467.256382  1.124751e+06
1  200001  553854.314402  7.305524e+05
2  200002  427859.249775  6.403460e+05
3  200003  326270.997287  4.379295e+05
4  200004  380530.430672  6.281208e+05


In [145]:
print("test_lower head :", test_lower[:3])
print("test_upper head :", test_upper[:3])
print("lower <= upper? :", np.all(test_lower <= test_upper))
print("寬度平均        :", (test_upper - test_lower).mean())

test_lower head : [822467.25638198 553854.31440239 427859.24977455]
test_upper head : [1124750.84586707  730552.40696786  640346.00007794]
lower <= upper? : True
寬度平均        : 219392.98410764078
